<a href="https://colab.research.google.com/github/kirtiibm/False-Data-injection/blob/Root-Directories/Water_Systems_Attack_Detection_FDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# numpy library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode

# keras
from keras.layers import Input, Dense
from keras.models import Model, load_model
from keras import optimizers
from keras.callbacks import *

# sklearn
from sklearn.metrics import mean_squared_error

# json
import json

# tensorflow
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

# sklearn library
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import MinMaxScaler

# others
import time
import os




In [ ]:
# import dataset from .csv to dataframe type (df)
dataset03_df = pd.read_csv(r'dataset03.csv')
dataset04_df = pd.read_csv(r'dataset04.csv')
test_dataset_df = pd.read_csv(r'test_dataset.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'dataset03.csv'

In [ ]:
def configure_plotly_browser_state(plotly_js_url='https://cdn.plot.ly/plotly-latest.min.js'):
    import IPython

    # Define JavaScript configuration
    js_config = f'''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({{
            paths: {{
              base: '/static/base',
              plotly: '{plotly_js_url}?noext',
            }},
          }});
        </script>
    '''

    # Display JavaScript configuration
    display(IPython.core.display.HTML(js_config))


In [ ]:
# get different color maps for all features in the dataset
pal = list(sns.color_palette(palette='Spectral', n_colors=dataset03_df.shape[1]).as_hex())+list(sns.color_palette(palette='Spectral', n_colors=dataset04_df.shape[1]).as_hex())
len(pal)

Model Presentation

### **Process Training Dataset** (Dataset03)


In [ ]:
# import the orginal dataset. df is "dataframe"
df_train_orig = pd.read_csv(r'dataset03.csv')
# get dates and columns with sensor readings
dates_train = df_train_orig['DATETIME']
sensor_cols = [col for col in df_train_orig.columns if col not in ['DATETIME','ATT_FLAG']]
# scale sensor data
scaler = MinMaxScaler()
X = pd.DataFrame(index = df_train_orig.index, columns = sensor_cols, data = scaler.fit_transform(df_train_orig[sensor_cols]))

# split into training and validation
X1, X2, _, _  = train_test_split(X, X, test_size=0.33, random_state=42)

### **Process 2 Testing Datasets** (Dataset04 and Test Dataset)

In [ ]:
# Load dataset with attacks
df_test_01 = pd.read_csv(r'dataset04.csv')
df_test_02 = pd.read_csv(r'test_dataset.csv')

# scale datasets
X3 = pd.DataFrame(index = df_test_01.index, columns = sensor_cols,
                  data = scaler.transform(df_test_01[sensor_cols]))
X4 = pd.DataFrame(index = df_test_02.index, columns = sensor_cols,
                  data = scaler.transform(df_test_02[sensor_cols]))

# get targets
Y3 = df_test_01['ATT_FLAG']
Y4 = df_test_02['ATT_FLAG']

In [ ]:
# classes
class AutoEncoder(object):
    """ Keras-based AutoEncoder (AE) class used for event detection.
        Attributes:
        params: dictionary with parameters defining the AE structure,
    """
    def __init__(self, **kwargs):
        """ Class constructor, stores parameters and initialize AE Keras model. """

        # Default parameters values. If nI is not given, the code will crash later.
        params = {
            'nI': None,
            'nH': 3,
            'cf': 1,
            'activation' : 'tanh',
            'optimizer' : None,
            'verbose' : 0
            }

        for key,item in kwargs.items():
            params[key] = item

        self.params = params

    def create_model(self):
        """ Creates Keras AE model.
            The model has nI inputs, nH hidden layers in the encoder (and decoder)
            and cf compression factor. The compression factor is the ratio between
            the number of inputs and the innermost hidden layer which stands between
            the encoder and the decoder. The size of the hidden layers between the
            input (output) layer and the innermost layer decreases (increase) linearly
            according to the cg.
        """

        # retrieve params
        nI = self.params['nI'] # number of inputs
        nH = self.params['nH'] # number of hidden layers in encoder (decoder)
        cf = self.params['cf'] # compression factor
        activation = self.params['activation'] # autoencoder activation fWeunction
        optimizer = self.params['optimizer'] # Keras optimizer
        verbose = self.params['verbose'] # echo on screen

        # get number/size of hidden layers for encoder and decoder
        temp = np.linspace(nI,nI/cf,nH + 1).astype(int)
        nH_enc = temp[1:]
        nH_dec = temp[:-1][::-1]

        # input layer placeholder
        input_layer = Input(shape=(nI,))

        # build encoder
        for i, layer_size in enumerate(nH_enc):
            if i == 0:
                # first hidden layer
                encoder = Dense(layer_size, activation=activation)(input_layer)
            else:
                # other hidden layers
                encoder = Dense(layer_size, activation=activation)(encoder)

        # build decoder
        for i, layer_size in enumerate(nH_dec):
            if i == 0:
                # first hidden layer
                decoder = Dense(layer_size, activation=activation)(encoder)
            else:
                # other hidden layers
                decoder = Dense(layer_size, activation=activation)(decoder)

        # create autoencoder
        autoencoder = Model(input_layer, decoder)
        if optimizer == None:
            optimizer = tf.optimizers.Adam(learning_rate = 0.001)

        # print autoencoder specs
        if verbose > 0:
            print('Created autoencoder with structure:');
            print(', '.join('layer_{}: {}'.format(v, i) for v, i in enumerate(np.hstack([nI,nH_enc,nH_dec]))))

        # compile and return model
        autoencoder.compile(optimizer=optimizer, loss='mean_squared_error')

        return autoencoder

    def train(self, x, **train_params):
        """ Train autoencoder,
            x: inputs (inputs == targets, AE are self-supervised ANN).
        """
        if self.params['verbose']:
            if self.ann == None:
                print('Creating model.')
                self.create_model()
        self.ann.fit(x, x, **train_params)


    def predict(self, x, test_params={}):
        """ Yields reconstruction error for all inputs,
            x: inputs.
        """
        return self.ann.predict(x, **test_params)

class AEED(AutoEncoder):
    """ This class extends the AutoEncoder class to include event detection
        functionalities.
    """
    def initialize(self):
        """ Create the underlying Keras model. """
        self.ann = self.create_model()

    def predict(self, x, **keras_params):
        """ Predict with autoencoder. """
        preds = pd.DataFrame(index=x.index,columns=x.columns,
                                            data=super(AEED, self).predict(x.values,keras_params))
        errors = (x-preds)**2
        return preds, errors

    def detect(self, x, theta, window = 1, average=False, sys_theta = 0, **keras_params):
        """ Detection performed based on (smoothed) reconstruction errors.
            x = inputs,
            theta = threshold, attack flagged if reconstruction error > threshold,
            window = length of the smoothing window (default = 1 timestep, i.e. no smoothing),
            average = boolean (default = False), if True the detection is performed
                on the average reconstruction error across all outputs,
            keras_params = parameters for the Keras-based AE prediction.
        """
        #        preds = super(AEED, self).predict(x,keras_params)
        preds, temp = self.predict(x, **keras_params)
        temp = (x-preds)**2
        if average:
            errors = temp.mean(axis=1).rolling(window=window).mean()
            detection = errors > theta
        else:
            errors = temp.rolling(window=window).mean()
            detection = errors.apply(lambda x: x>np.max(theta.name, sys_theta))

        return detection, errors

    def save(self, filename):
        """ Save AEED model.
            AEED parameters saved in a .json, while Keras model is stored in .h5 .
        """
        # parameters
        with open(filename+'.json', 'w') as fp:
            json.dump(self.params, fp)
        # keras model
        self.ann.save(filename+'.h5')
        # echo
        print('Saved AEED parameters to {0}.\nKeras model saved to {1}'.format(filename+'.json', filename+'.h5'))


# functions
def load_AEED(params_filename, model_filename):
    """ Load stored AEED. """
    # load params and create AEED
    with open(params_filename) as fd:
        params = json.load(fd)
    aeed = AEED(**params)

    # load keras model
    aeed.ann = load_model(model_filename)
    return aeed

In [ ]:
# define model parameters
params = {
    'nI' : X.shape[1],
    'nH' : 3,
    'cf' : 2.5,
    'activation' : 'tanh',
    'verbose' : 1,
}

# create AutoEncoder for Event Detection (AEED)
autoencoder = AEED(**params)
autoencoder.initialize()

In [ ]:
# train models with early stopping and reduction of learning rate on plateau
earlyStopping= EarlyStopping(monitor='val_loss', patience=3, verbose=0,  min_delta=1e-4, mode='auto')
lr_reduced = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, verbose=0, epsilon=1e-4, mode='min')

# train autoencoder
autoencoder.train(X1.values,
            epochs=500,                              # train for the maximum times of 500 times
            batch_size=40,                           # learning-material "chunks" size
            shuffle=False,                           # don't mix the material randomly
            callbacks = [earlyStopping, lr_reduced], # training optimization methods
            verbose = 2,                             # provide me 1 line of results per epoch
            validation_data=(X2.values, X2.values))  # after every train, test yourself

In [ ]:
# assess detection
def compute_scores(Y,Yhat):
    return [accuracy_score(Y,Yhat),f1_score(Y,Yhat),precision_score(Y,Yhat),recall_score(Y,Yhat)]
    # get validation reconstruction errors
_, validation_errors = autoencoder.predict(X2)

# set treshold as quantile of average reconstruction error
theta = validation_errors.mean(axis = 1).quantile(0.995)

Yhat3, _ = autoencoder.detect(X3, theta = theta , window = 3, average=True)
Yhat4, _ = autoencoder.detect(X4, theta = theta, window = 3, average=True)

In [ ]:
results = pd.DataFrame(index = ['test dataset 01','test dataset 02'],
                       columns = ['accuracy','f1_score','precision','recall'])
results.loc['test dataset 01'] = compute_scores(Y3,Yhat3)
results.loc['test dataset 02'] = compute_scores(Y4,Yhat4)

print('Results:\n')
print(results)

In [ ]:
# plot figures
shade_of_gray = '0.75'
f, axes = plt.subplots(2,figsize = (20,8))
axes[0].plot(Yhat3, color = shade_of_gray, label = 'predicted state')
axes[0].fill_between(Yhat3.index, Yhat3.values, where=Yhat3.values <=1, interpolate=True, color=shade_of_gray)
axes[0].plot(Y3, color = 'r', alpha = 0.85, lw = 5, label = 'real state')
axes[0].set_title('Detection trajectory on test dataset 01', fontsize = 14)
axes[0].set_yticks([0,1])
axes[0].set_yticklabels(['NO ATTACK','ATTACK'])
axes[0].legend(fontsize = 12, loc = 2)

axes[1].plot(Yhat4, color = shade_of_gray, label = 'predicted state')
axes[1].fill_between(Yhat4.index, Yhat4.values, where=Yhat4.values <=1, interpolate=True, color=shade_of_gray)
axes[1].plot(Y4, color = 'r', alpha = 0.75, lw = 5, label = 'real state')
axes[1].set_title('Detection trajectory on test dataset 02', fontsize = 14)
axes[1].set_yticks([0,1])
axes[1].set_yticklabels(['NO ATTACK','ATTACK'])

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Compute confusion matrix for test dataset 01
cm1 = confusion_matrix(Y3, Yhat3)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm1, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.title("Confusion Matrix - Test Dataset 01")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# Compute confusion matrix for test dataset 02
cm2 = confusion_matrix(Y4, Yhat4)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm2, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.title("Confusion Matrix - Test Dataset 02")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# Define function to extract TP, FP, TN, FN from confusion matrix
def extract_confusion_values(cm):
    TN = cm[0, 0]
    FP = cm[0, 1]
    FN = cm[1, 0]
    TP = cm[1, 1]
    return TP, FP, TN, FN

# Extract values for test dataset 01
TP1, FP1, TN1, FN1 = extract_confusion_values(cm1)

# Plot for test dataset 01
plt.figure(figsize=(10, 6))
barWidth = 0.3
labels = ['True Negative', 'False Positive', 'False Negative', 'True Positive']

# Position of bars on x-axis
r1 = np.arange(len(labels))

# Make the plot
plt.bar(r1, [TN1, FP1, FN1, TP1], color=['#2d7f5e', '#ff7f0e', '#d62728', '#9467bd'])

# Add labels
plt.xlabel('Metrics', fontweight='bold')
plt.xticks([r for r in range(len(labels))], labels)
plt.ylabel('Count', fontweight='bold')
plt.title('Confusion Matrix Breakdown - Test Dataset 01')
plt.show()

# Extract values for test dataset 02
TP2, FP2, TN2, FN2 = extract_confusion_values(cm2)

# Plot for test dataset 02
plt.figure(figsize=(10, 6))

# Make the plot
plt.bar(r1, [TN2, FP2, FN2, TP2], color=['#2d7f5e', '#ff7f0e', '#d62728', '#9467bd'])

# Add labels
plt.xlabel('Metrics', fontweight='bold')
plt.xticks([r for r in range(len(labels))], labels)
plt.ylabel('Count', fontweight='bold')
plt.title('Confusion Matrix Breakdown - Test Dataset 02')
plt.show()


In [ ]:
tensorflow.keras.models.save_model(model, 'model.pbtxt')
converter = tensorflow.lite.TFliteConverter.from_keras_model(model = model)
model_tflite = converter.convert()
open("FDIattack.tflite", "wb").write(model_tflite) # Removed extra indentation on this line